In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint

In [2]:
headers = {"Accept-Language":"en-US,en;q=0.5"}

In [5]:
movie_name = []
year = []
time = []
rating = []
metascore = []
votes = []
gross = []
description = []

In [6]:
pages = np.arange(1,1000,100)

In [7]:
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text,'html.parser')
    movie_data = soup.findAll('div',attrs ={'class':'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_relese = store.h3.find('span',class_ = 'lister-item-year text-muted unbold').text
        year.append(year_of_relese)
        
        runtime = store.p.find("span", class_ = 'runtime').text
        time.append(runtime)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '')
        rating.append(rate)
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else "****"
        metascore.append(meta)
        
        value = store.find_all('span', attrs = {'name': "nv"}) # both vote and gross have same attribute called name as nv.
        
        vote = value[0].text
        votes.append(vote)
        
        grosses = value[1].text if len(value)>1 else '%^%^%^'
        gross.append(grosses)
        
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
        description.append(description_)


In [8]:
movie_list = pd.DataFrame({ "Movie Name": movie_name, "Year of Release" : year, "Watch Time": time,"Movie Rating": rating, "Meatscore of movie": metascore, "Votes" : votes, "Gross": gross, "Description": description  })

In [9]:
movie_list.head(5)

,Movie Name,Year of Release,Watch Time,Movie Rating,Meatscore of movie,Votes,Gross,Description
0,The Shawshank Redemption,(1994),142 min,9.3,82,"2,791,412",$28.34M,"Over the course of several years, two convicts..."
1,The Godfather,(1972),175 min,9.2,100,"1,944,251",$134.97M,"Don Vito Corleone, head of a mafia family, dec..."
2,The Dark Knight,(2008),152 min,9.0,84,"2,771,584",$534.86M,When the menace known as the Joker wreaks havo...
3,Schindler's List,(1993),195 min,9.0,95,"1,403,825",$96.90M,"In German-occupied Poland during World War II,..."
4,The Godfather Part II,(1974),202 min,9.0,90,"1,320,883",$57.30M,The early life and career of Vito Corleone in ...


In [10]:
# #saving the data in excel format
movie_list.to_excel("Top 1000 IMDb movies.xlsx")

# #If you want to save the data in csv format
movie_list.to_csv("Top 1000 IMDb movies.csv")